In [ ]:
# Install Kaggle API
!pip install kaggle

# Upload kaggle.json
from google.colab import files
files.upload()  # Upload kaggle.json

# Setup Kaggle API credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download Sentiment140 dataset
!kaggle datasets download -d kazanova/sentiment140
!unzip sentiment140.zip

# Save to sentiment140_clean.csv
import pandas as pd
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='ISO-8859-1', header=None)
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df.to_csv("sentiment140_clean.csv", index=False)